In [62]:
import pandas as pd
import numpy as np

In [23]:
df = pd.read_csv('emojis.csv', encoding='utf-8',error_bad_lines=False,header=None)
df.columns = ['emoji','text']

b'Skipping line 238: expected 2 fields, saw 3\nSkipping line 239: expected 2 fields, saw 3\nSkipping line 241: expected 2 fields, saw 3\nSkipping line 242: expected 2 fields, saw 3\nSkipping line 244: expected 2 fields, saw 4\nSkipping line 245: expected 2 fields, saw 4\nSkipping line 246: expected 2 fields, saw 5\nSkipping line 247: expected 2 fields, saw 5\nSkipping line 248: expected 2 fields, saw 5\nSkipping line 249: expected 2 fields, saw 4\nSkipping line 250: expected 2 fields, saw 4\nSkipping line 251: expected 2 fields, saw 5\nSkipping line 252: expected 2 fields, saw 5\nSkipping line 253: expected 2 fields, saw 5\nSkipping line 254: expected 2 fields, saw 4\nSkipping line 255: expected 2 fields, saw 4\nSkipping line 256: expected 2 fields, saw 5\nSkipping line 257: expected 2 fields, saw 5\nSkipping line 258: expected 2 fields, saw 5\nSkipping line 259: expected 2 fields, saw 3\nSkipping line 260: expected 2 fields, saw 4\nSkipping line 261: expected 2 fields, saw 3\nSkipping

# 학습

1. 그냥 자기들끼리 학습 word2vec => 평균 (300,)
2. 그냥 자기들끼리 학습 doc2vec => 평균(300,)
3. bert uncased pretrained 모델 사용 => (768,)
4. w2v finetuned => 사전에 없는 단어 있어서 실패! 

In [29]:
df['text_pre'] = df.text.str.lower()
df['text_pre'] = df['text_pre'].str.replace(pat='"', repl='', regex=False)
df

In [35]:
# 학습시킬 text
result = [[word for word in sen.split()] for sen in df.text_pre]
len(result)

1511

In [72]:
# 1. 그냥 자기들끼리 학습 word2vec => 평균
from gensim.models import Word2Vec
w2v_model = Word2Vec(sentences=result, size=300, window=5, min_count=1, workers=4, sg=1) # 학습한 거 기억해~
w2v_model.wv.vectors.shape

(1476, 300)

In [ ]:
# 2. 그냥 자기들끼리 학습 doc2vec => 평균
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(result)]
d2v_model = Doc2Vec(documents, vector_size=300, window=5, min_count=1, workers=4)
d2v_model.wv.vectors.shape

In [60]:
# np.mean(model.wv[['grinning', 'face']], axis=0).shape : (300,)
def cal_embed(text,model):
    return np.mean(model.wv[text.split()], axis=0)

In [79]:
df['w2v_self'] = df.text_pre.apply(lambda x: cal_embed(x,w2v_model))
df['d2v_self'] = df.text_pre.apply(lambda x: cal_embed(x,d2v_model))
df.head()

,emoji,text,text_pre,w2v_self,d2v_self
0,😀,"""Grinning Face""",grinning face,"[-0.00094045, -0.0006872087, -6.5182445e-05, 0...","[-0.0009106955, -0.00066791294, -5.793549e-05,..."
1,😁,"""Beaming Face With Smiling Eyes""",beaming face with smiling eyes,"[-0.0006431065, 9.477781e-05, 0.00034368213, -...","[-0.0005997297, 0.0001040432, 0.00033933454, -..."
2,😂,"""Face With Tears of Joy""",face with tears of joy,"[0.00019971476, -0.0004615941, -0.00011864512,...","[0.00023946613, -0.00043663365, -0.00010684356..."
3,🤣,"""Rolling on the Floor Laughing""",rolling on the floor laughing,"[0.00030254797, -0.00021019144, -0.00015853446...","[0.0003178202, -0.00019955196, -0.00015189362,..."
4,😃,"""Grinning Face With Big Eyes""",grinning face with big eyes,"[-0.0003975432, -0.0003180963, 0.00035707135, ...","[-0.00035680766, -0.000299274, 0.00035843992, ..."


# dataload

In [85]:
df.to_pickle('emoji_w2v.pkl')

In [89]:
df = pd.read_pickle('emoji_w2v.pkl')
df

,emoji,text,text_pre,w2v_self,d2v_self,bert_token_enc
0,😀,"""Grinning Face""",grinning face,"[-0.00094045, -0.0006872087, -6.5182445e-05, 0...","[-0.0009106955, -0.00066791294, -5.793549e-05,...","[-0.74729633, -0.087494224, 0.63072586, 0.4789..."
1,😁,"""Beaming Face With Smiling Eyes""",beaming face with smiling eyes,"[-0.0006431065, 9.477781e-05, 0.00034368213, -...","[-0.0005997297, 0.0001040432, 0.00033933454, -...","[-0.88575333, -0.1869557, -0.3815595, 0.729069..."
2,😂,"""Face With Tears of Joy""",face with tears of joy,"[0.00019971476, -0.0004615941, -0.00011864512,...","[0.00023946613, -0.00043663365, -0.00010684356...","[-0.7234129, -0.1108921, 0.44371545, 0.5689685..."
3,🤣,"""Rolling on the Floor Laughing""",rolling on the floor laughing,"[0.00030254797, -0.00021019144, -0.00015853446...","[0.0003178202, -0.00019955196, -0.00015189362,...","[-0.724951, -0.08921164, 0.52543116, 0.5329707..."
4,😃,"""Grinning Face With Big Eyes""",grinning face with big eyes,"[-0.0003975432, -0.0003180963, 0.00035707135, ...","[-0.00035680766, -0.000299274, 0.00035843992, ...","[-0.92428726, -0.19967505, -0.5458071, 0.81853..."
...,...,...,...,...,...,...
1506,🇿🇲,"""Zambia""",zambia,"[-0.0005835779, -0.00027042735, 0.00063267356,...","[-0.0005835779, -0.00027042735, 0.00063267356,...","[-0.70810485, -0.16304275, 0.56932163, 0.47378..."
1507,🇿🇼,"""Zimbabwe""",zimbabwe,"[0.0011473428, -0.00047775332, -0.0015731562, ...","[0.0011473428, -0.00047775332, -0.0015731562, ...","[-0.7186055, -0.17344351, 0.5435216, 0.517652,..."
1508,🏴󠁧󠁢󠁥󠁮󠁧󠁿,"""England""",england,"[-0.0006612721, -0.0008092607, 0.0013948333, 0...","[-0.0006612721, -0.0008092607, 0.0013948333, 0...","[-0.7511741, -0.16930981, 0.65287125, 0.499871..."
1509,🏴󠁧󠁢󠁳󠁣󠁴󠁿,"""Scotland""",scotland,"[-0.00060080085, 0.0012818298, -0.00058121135,...","[-0.00060080085, 0.0012818298, -0.00058121135,...","[-0.70290583, -0.12438569, 0.6124369, 0.439223..."
